# Slimming down cleaning process and make it able to take in any new data
##### Started 14.12.22

### Imports

In [2]:
import pandas as pd
import random
import string
import re
import json
import ast
from collections import Counter


### Getting workable DF

In [3]:
data_path = '~/code/emilycardwell/final-project-data/data/raw/kaggle_raw.csv'
raw_df = pd.read_csv(data_path)
raw_df.head(3)

,Unnamed: 0,artist_name,song_name,chords&lyrics,chords,lyrics,tabs,lang,artist_id,followers,genres,popularity,name_e_chords
0,0,Justin Bieber,"10,000 Hours",\nCapo on 3rd fret\n\t \t\t \r\n\r\nVerse 1:\...,{3: 'G G/B ...,"{0: '\nCapo on 3rd fret\n\t \t\t ', 1: '', 2:...","{89: ""I-----, I'm gonna love you ""}",en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
1,1,Justin Bieber,2 Much,\n\t \t\t\r\nIntro: F#m7 D2 \r\n\r\nVerse 1:...,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","{0: '\n\t \t\t', 2: '', 3: 'Verse 1:', 5: ' ...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber
2,2,Justin Bieber,2u (feat. David Guetta),\n\t \t\t\r\nEm D C ...,{1: 'Em D C C...,"{0: '\n\t \t\t', 2: ""No limit in the sky that...",{},en,1uNFoZAHBGtllmzznpCI3s,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",100,justin-bieber


In [4]:
slim_raw_df = raw_df[['artist_name', 'song_name', 'chords', 'genres', 'popularity']]
slim_raw_df.head(3)

,artist_name,song_name,chords,genres,popularity
0,Justin Bieber,"10,000 Hours",{3: 'G G/B ...,"['canadian pop', 'pop', 'post-teen pop']",100
1,Justin Bieber,2 Much,"{1: 'Intro: F#m7 D2 ', 4: 'F#m7 ', 8: 'D2 ', ...","['canadian pop', 'pop', 'post-teen pop']",100
2,Justin Bieber,2u (feat. David Guetta),{1: 'Em D C C...,"['canadian pop', 'pop', 'post-teen pop']",100


In [14]:
split_raw_df = slim_raw_df.copy()
splits = []

for row in split_raw_df.chords:
    row_dict = ast.literal_eval(row)
    for key, value in row_dict.items():
        r_or = value.split('|')[0]
        etc1 = r_or.replace('.', ' ')
        ect2 = etc1.replace('*', '')
        p = re.sub(r'[()]', '', ect2)
        s = re.sub(r'^\s|\s$', '', p)
        row_dict[key] = re.sub(r'\s+|-|~|,', ' ', s)
    splits.append(row_dict)

split_raw_df.chords = splits

In [9]:
# r = random.randint(0, 135782)
# slim_raw_df.chords[r]

In [10]:
# split_raw_df.chords[r]

#### See Chord Frequency

In [11]:
chords_counter = Counter()
for song in split_raw_df.chords:
    for value in song.values():
        l = value.split(' ')
        chords_counter.update(l)
del chords_counter['']

# slim_chord_counts_dict = {}
# for chord, count in chords_count_dict.items():
#     if count <= low_freq_to_remove:
#         pass
#     else:
#         slim_chord_counts_dict[chord] = count

chord_count_df = pd.Series(dict(chords_counter)).to_frame('chord_count')
sorted_cc_df = chord_count_df.sort_values(by='chord_count', ascending=False)
len(sorted_cc_df)

21727

In [13]:
for chord in sorted_cc_df.index:
    if chord[:2] == 'Bb':
        print(chord)

Bb
Bbm
Bb7
Bbm7
Bb9
Bb7M
Bb5
Bb/D
Bb7+
Bbmaj7
Bb/C
Bb6
Bb/F
Bb7/9
Bbº
Bbm6
Bb4
Bbsus4
Bbdim
Bb/A
Bbsus2
Bb2
Bb7/13
Bb/Ab
Bbm5
Bb79
Bbsus
Bb/G
Bb/Eb
Bbm9
Bb°
Bbadd9
Bb713
Bbm/F
Bbm/Db
Bb6/9
Bb11
Bb7/4
Bb7M9
Bbm/Ab
Bb59
Bbmaj9
Bbm79
Bbdim7
Bb13
Bb5+
Bb74/9
Bb7sus4
Bbadd2
Bb7/D
Bb/E
Bbm5+
Bbm6/Db
Bb9/D
Bb7M/F
Bbm7/5
Bbmaj7/F
Bb/B
Bb7#11
BbM7
Bbm7/Ab
Bb4/7
Bbm7/G#
Bb7/F
Bb7M/D
Bbm/F#
Bb4/7/9
Bbo
Bbm/G#
Bb/G#
Bb9/C
Bbm7b5
Bbaug
Bb7b13
Bb7/11+
Bbm7+
Bb79/#11
Bbm7/9
Bb7b5
Bb6/F
Bbm75b
Bb7/Ab
Bb7b9
Bb911+
Bb~C~Bb~C
Bb69
Bb7/C
Bb4/F#
Bbm7M
Bbm7/Eb
Bbm7/F
Bbm7/4
Bb/Db
Bb74
Bbsus9
Bb7/5
Bb9/G
Bb#11
Bbm/A
Bb7+/9
Bb7/49
Bb97
Bbm4
Bbsus4/F
Bb\D
Bbm/C#
Bb7sus
Bb711
Bb7/6
Bb7/5+
BbF
Bb7#5
Bb711+
Bbsus7
Bb2/F
Bb7/9+
Bbm/Eb
Bbm711
Bb9/F#
Bbm7add4
Bbm7/11
Bb7+/F
Bb7M/A
Bbsus4/EbEb
Bbm75
Bb11/D
Bbm65+
Bb7/4/9
Bb2/D
BbM
Bb5/Ab
Bb7#9
Bbm/Gb
Bbmaj
Bbm/G
Bbm/C
Bbm6/F
Bb6/D
Bbmaj7/A
Bbmmaj7
Bb9/F
Bb7M/9
Bbno5
Bbm6/C
Bbx2
Bbm/B
Bb/C#
Bb6/5
Bb79/13
Bb/F#
Bb7/11
Bb11+
Bb\F
Bbsus4/7/9
Bb79/11
BbÂ°
Bbm713
Bb5/F
Bb7

## Creating List of Correctly Formatted Chords

In [16]:
major_chords = ['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B']

In [20]:
chords_df = pd.DataFrame(index=['m', 'dim', 'aug', '5', 'aug5', '5b7',
                                'dim7', 'hdim7', 'm7', '7', 'M7',  
                                'm9', '9', '7b9',  'M9', '11',
                                'add2', 'add4', 'add6', 'add9', 'madd2', 'madd4', 'madd6', 'madd9', 
                                'msus2', 'msus4', '7sus2', '7sus4', 'sus2', 'sus4', 
                                'm/2', 'm/b3', 'm/3', 'm/4', 'm/5', 'm/6','m/b7', 'm/7',
                                'm7/b3', 'm7/4', 'm7/5', 'm7/M7',
                                '7/2', '7/m3', '7/3', '7/4', '7/5', '7/6','7/M7', 
                                '9/b3', '9/3', '9/4', '9/5', '9/6', '9/M7',
                                '/2', '/b3', '/3', '/4', '/5', '/6','/b7', '/7', 
                                'M7/2', 'M7/b3', 'M7/3', 'M7/4', 'M7/5', 'M7/6','M7/b7',
                                ],
                         columns=major_chords)


In [21]:
for idx in chords_df.index:
    new_chords = [m + idx for m in major_chords]
    chords_df.loc[idx, :] = new_chords
chords_df

,C,C#,Db,D,D#,Eb,E,F,F#,Gb,G,G#,Ab,A,A#,Bb,B
m,Cm,C#m,Dbm,Dm,D#m,Ebm,Em,Fm,F#m,Gbm,Gm,G#m,Abm,Am,A#m,Bbm,Bm
dim,Cdim,C#dim,Dbdim,Ddim,D#dim,Ebdim,Edim,Fdim,F#dim,Gbdim,Gdim,G#dim,Abdim,Adim,A#dim,Bbdim,Bdim
aug,Caug,C#aug,Dbaug,Daug,D#aug,Ebaug,Eaug,Faug,F#aug,Gbaug,Gaug,G#aug,Abaug,Aaug,A#aug,Bbaug,Baug
5,C5,C#5,Db5,D5,D#5,Eb5,E5,F5,F#5,Gb5,G5,G#5,Ab5,A5,A#5,Bb5,B5
aug5,Caug5,C#aug5,Dbaug5,Daug5,D#aug5,Ebaug5,Eaug5,Faug5,F#aug5,Gbaug5,Gaug5,G#aug5,Abaug5,Aaug5,A#aug5,Bbaug5,Baug5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M7/3,CM7/3,C#M7/3,DbM7/3,DM7/3,D#M7/3,EbM7/3,EM7/3,FM7/3,F#M7/3,GbM7/3,GM7/3,G#M7/3,AbM7/3,AM7/3,A#M7/3,BbM7/3,BM7/3
M7/4,CM7/4,C#M7/4,DbM7/4,DM7/4,D#M7/4,EbM7/4,EM7/4,FM7/4,F#M7/4,GbM7/4,GM7/4,G#M7/4,AbM7/4,AM7/4,A#M7/4,BbM7/4,BM7/4
M7/5,CM7/5,C#M7/5,DbM7/5,DM7/5,D#M7/5,EbM7/5,EM7/5,FM7/5,F#M7/5,GbM7/5,GM7/5,G#M7/5,AbM7/5,AM7/5,A#M7/5,BbM7/5,BM7/5
M7/6,CM7/6,C#M7/6,DbM7/6,DM7/6,D#M7/6,EbM7/6,EM7/6,FM7/6,F#M7/6,GbM7/6,GM7/6,G#M7/6,AbM7/6,AM7/6,A#M7/6,BbM7/6,BM7/6


In [ ]:
sharp_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 'C']
flats_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B', 'C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B', 'C']
sharp_keys = ['D', 'E', 'G', 'A', 'B', 'Dm', 'Em', 'Fm', 'Bm']
flats_keys = ['C', 'F', 'Cm', 'Gm', 'Am']

In [ ]:
# big function
def get_slash_notes(chords):
    slashes = []
    i = 0 
    
    def string_appending(chord, notes, i):
        s2 = f'{chord}/{notes[i+2]}'
        slashes.append(s2)
        sm3 = f'{chord}/{notes[i+3]}'
        slashes.append(sm3)
        s3 = f'{chord}/{notes[i+4]}'
        slashes.append(s3)
        s4 = f'{chord}/{notes[i+5]}'
        slashes.append(s4)
        s5 = f'{chord}/{notes[i+7]}'
        slashes.append(s5)
        sm6 = f'{chord}/{notes[i+8]}'
        slashes.append(sm6)
        s6 = f'{chord}/{notes[i+9]}'
        slashes.append(s6)
        sm7 = f'{chord}/{notes[i+10]}'
        slashes.append(sm7)
        s7 = f'{chord}/{notes[i+11]}'
        slashes.append(s7)
        
    for chord in chords:
        
        for sh in sharp_keys:
            if sh in chord :
                i = sharp_notes.index(chord[0])
                string_appending(chord, sharp_notes, i)
            elif len(chord) > 1:
                if '#' == chord[1]:
                    i = sharp_notes.index(chord[:2])
                    string_appending(chord, sharp_notes, i)

        for fl in flats_keys:
            if fl in chord:
                i = flats_notes.index(chord[0])
                string_appending(chord, flats_notes, i)
            elif len(chord) > 1:
                if 'b' in chord:
                    i = flats_notes.index(chord[:2])
                    string_appending(chord, flats_notes, i)
                
            
    return slashes

slash_chords = get_slash_notes(extra_chords)

In [ ]:
final_chords = extra_chords + slash_chords
len(final_chords)